In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, to_date, to_timestamp, current_date, current_timestamp, date_format
from pyspark.sql.types import StructType, StructField, DoubleType, DateType, FloatType

# SparkSession
spark = SparkSession.builder \
    .appName("deltatable") \
    .getOrCreate()

# Define schema for the data
schema = StructType([
    StructField("Date/Time", DateType(), True),
    StructField("LV_ActivePower", FloatType(), True),
    StructField("Wind_Speed", FloatType(), True),
    StructField("Theoretical_Power_Curve", FloatType(), True),
    StructField("Wind_Direction", FloatType(), True)
])

# Define Kafka source
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "sparkread") \
    .option("startingOffsets", "earliest") \
    .load()

# Check if DataFrame is streaming
if df.isStreaming:
    print("\nDataFrame is streaming. Monitoring for new data...\n")
else:
    print("\nDataFrame is not streaming. No new data to monitor.\n")

# Parse JSON data and select relevant columns
json_df = df.selectExpr("CAST(value AS STRING)") \
            .select(from_json(col("value"), schema).alias("data")) \
            .select("data.*")

# Apply transformations
update_df = json_df.withColumn("signal_date", to_date(col("Date/Time"), "yyyy-MM-dd")) \
                   .withColumn("signal_tc", to_timestamp(col("Date/Time"), "yyyy-MM-dd HH:mm:ss")) \
                   .withColumn("create_date", date_format(current_date(), 'dd-MM-yyyy')) \
                   .withColumn("create_ts", date_format(current_timestamp(), 'dd-MM-yyyy HH:mm:ss'))

# Print schema of the updated DataFrame
update_df.printSchema()


In [ ]:
from pyspark.sql.functions import create_map, lit

# Define the signal names
signal_names = ['LV_ActivePower', 'Wind_Speed', 'Theoretical_Power_Curve', 'Wind_Direction']

# Create a map column with signal names and their corresponding values
map_expr = create_map(*[lit(signal).alias(signal) for signal in signal_names])

# Add the map column to the DataFrame
update_df = update_df.withColumn("signal", map_expr)

# Print schema of the updated DataFrame
update_df.printSchema()


In [ ]:
from pyspark.sql.functions import create_map, lit

# Define the signal names and corresponding column names
signal_names = ['LV ActivePower', 'Wind Speed', 'Theo_Power_Curve', 'Wind Direction']
column_names = ['LV_ActivePower', 'Wind_Speed', 'Theoretical_Power_Curve', 'Wind_Direction']

# Create a map column with signal names as keys and corresponding column names as values
map_expr = create_map([lit(signal).alias(signal) for signal in signal_names],
                      [lit(column).alias(column) for column in column_names])

# Add the map column to the DataFrame
update_df = update_df.withColumn("signal", map_expr)

# Print schema of the updated DataFrame
update_df.printSchema()


In [ ]:
[
    {
        "sig_name":"LV ActivePower",
        "sig_mapping_name":"LV ActivePower_average"
    },
    {
        "sig_name":"Wind Speed",
        "sig_mapping_name":"Wind Speed_average"
    },
    {
        "sig_name":"Theoretical_Power_Curve",
        "sig_mapping_name":"Theoretical_Power_Curve_average"
    },
    {
        "sig_name":"Wind Direction",
        "sig_mapping_name":"Wind Direction_average"
    },
    {
        "sig_name":"Wind Direction",
        "sig_mapping_name":"Wind Direction_average"
    },
]

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Create SparkSession
spark = SparkSession.builder \
    .appName("CreateDataFrameFromJSON") \
    .getOrCreate()

# JSON data
json_data = [
    {"sig_name": "LV ActivePower", "sig_mapping_name": "LV ActivePower_average"},
    {"sig_name": "Wind Speed", "sig_mapping_name": "Wind Speed_average"},
    {"sig_name": "Theoretical_Power_Curve", "sig_mapping_name": "Theoretical_Power_Curve_average"},
    {"sig_name": "Wind Direction", "sig_mapping_name": "Wind Direction_average"},
    {"sig_name": "Wind Direction", "sig_mapping_name": "Wind Direction_average"}
]

# Create DataFrame from JSON data
df = spark.createDataFrame([Row(**x) for x in json_data])

# Show the DataFrame
df.show()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import broadcast

# Create SparkSession
spark = SparkSession.builder \
    .appName("ChangeSignalNames") \
    .getOrCreate()

# JSON data for signal mapping
json_data = [
    {"sig_name": "LV ActivePower", "sig_mapping_name": "LV ActivePower_average"},
    {"sig_name": "Wind Speed", "sig_mapping_name": "Wind Speed_average"},
    {"sig_name": "Theoretical_Power_Curve", "sig_mapping_name": "Theoretical_Power_Curve_average"},
    {"sig_name": "Wind Direction", "sig_mapping_name": "Wind Direction_average"},
    {"sig_name": "Wind Direction", "sig_mapping_name": "Wind Direction_average"}
]

# Create DataFrame from JSON data
mapping_df = spark.createDataFrame([Row(**x) for x in json_data])

# Perform broadcast join
broadcast_df = df.join(broadcast(mapping_df),
                       df.generation_indicator == mapping_df.sig_name,
                       "left_outer")

# Update the 'generation_indicator' column
broadcast_df = broadcast_df.withColumn("generation_indicator", broadcast_df.sig_mapping_name)

# Drop unnecessary columns
broadcast_df = broadcast_df.drop("sig_name", "sig_mapping_name")

# Show the updated DataFrame
broadcast_df.show()


Exmaple


In [ ]:
from pyspark.sql import SparkSession
import random

# Create SparkSession
spark = SparkSession.builder \
    .appName("FakeDataFrame") \
    .getOrCreate()

# Fake data
fake_data = [
    {"name": "Alice", "id": 1, "salary": 50000},
    {"name": "Bob", "id": 2, "salary": 60000},
    {"name": "Charlie", "id": 3, "salary": 55000},
    {"name": "David", "id": 4, "salary": 70000},
    {"name": "Eve", "id": 5, "salary": 65000}
]

# Create DataFrame
df = spark.createDataFrame(fake_data)

# Show the DataFrame
df.show()


In [ ]:
# Filter DataFrame to select rows where 'id' is greater than 3
filtered_df = df.filter(df['id'] > 3)

# Select the 'id' column from the filtered DataFrame
filtered_ids = filtered_df.select('id')

# Show the values of the 'id' column
filtered_ids.show()


In [ ]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
    .appName("DataFrameSQL") \
    .getOrCreate()

# Create a DataFrame with some data
data = [("Alice", 1, 50000),
        ("Bob", 2, 60000),
        ("Charlie", 3, 55000),
        ("David", 4, 70000),
        ("Eve", 5, 65000)]
columns = ["name", "id", "salary"]
df = spark.createDataFrame(data, columns)

# Register the DataFrame as a temporary view
df.createOrReplaceTempView("employee")

# Perform SQL operations
sql_result = spark.sql("SELECT * FROM employee WHERE id > 3")

# Show the result
sql_result.show()


In [ ]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
    .appName("DataFrameSQL") \
    .getOrCreate()

# Create a DataFrame with some data
data = [("Alice", 1, 50000),
        ("Bob", 2, 60000),
        ("Charlie", 3, 55000),
        ("David", 4, 70000),
        ("Eve", 5, 65000)]
columns = ["name", "id", "salary"]
df = spark.createDataFrame(data, columns)

# Register the DataFrame as a temporary view
df.createOrReplaceTempView("employee")

# Perform SQL operation to select rows where 'id' and 'salary' are greater than 2 and 55000 respectively
sql_result = spark.sql("SELECT * FROM employee WHERE id > 2 AND salary > 55000")

# Show the result
sql_result.show()
